# Models

前面都是使用的`AutoModel`，这是一个智能的wrapper，可以根据你给定的checkpoint名字，自动去寻找对应的网络结构，故名Auto。

如果明确知道我们需要的是什么网络架构，就可以直接使用具体的`*Model`，比如`BertModel`，就是使用Bert结构。

## 随机初始化一个Transformer模型：通过`config`来加载

In [2]:
from transformers import BertModel, BertConfig

config = BertConfig()
model = BertModel(config)  # 模型是根据config来构建的，这时构建的模型是参数随机初始化的

In [3]:
print(config)

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.3.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



更常用的做法则是直接加载预训练模型，然后微调。

## 初始化一个预训练的Transformer模型：通过`from_pretrained`来加载

In [4]:
from transformers import BertModel

model = BertModel.from_pretrained('bert-base-cased')

模型的保存：

In [5]:
model.save_pretrained("directory_on_my_computer")
# 会生成两个文件： config.json pytorch_model.bin

# Tokenizer
transformer模型使用的分词方法，往往不是直接的word-level分词或者char-level分词。\
前者会让词表过大，后者则表示能力很低。\
因此主流的方式是进行 subword-level 的分词。例如对 "tokenization" 这个词，可能会被分成 "token" 和 "ization" 两部分。

常见的subword tokenization方法有：
- BPE
- WordPiece
- Unigram
- ...


In [6]:
from transformers import BertTokenizer  # 或者 AutoTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

In [9]:
s = 'today is a good day to learn transformers'
tokenizer()

{'input_ids': [101, 2052, 1110, 170, 1363, 1285, 1106, 3858, 11303, 1468, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

## 了解一下内部的具体步骤：

1. `tokenize()`

In [12]:
s = 'today is a good day to learn transformers'
tokens = tokenizer.tokenize(s)
tokens

['today', 'is', 'a', 'good', 'day', 'to', 'learn', 'transform', '##ers']

2. `convert_tokens_to_ids()`

In [13]:
ids = tokenizer.convert_tokens_to_ids(tokens)
ids

[2052, 1110, 170, 1363, 1285, 1106, 3858, 11303, 1468]

3. `decode`

In [16]:
print(tokenizer.decode([1468]))
print(tokenizer.decode(ids))  # 注意这里会把subword自动拼起来

##ers
today is a good day to learn transformers


## Special Tokens

观察一下上面的结果，直接call tokenizer得到的ids是：
```
[101, 2052, 1110, 170, 1363, 1285, 1106, 3858, 11303, 1468, 102]
```
而通过`convert_tokens_to_ids`得到的ids是：
```
[2052, 1110, 170, 1363, 1285, 1106, 3858, 11303, 1468]
```
可以发现，前者在头和尾多了俩token，id分别是 101 和 102。

decode出来瞅瞅：

In [17]:
tokenizer.decode([101, 2052, 1110, 170, 1363, 1285, 1106, 3858, 11303, 1468, 102])

'[CLS] today is a good day to learn transformers [SEP]'

它们分别是 `[CLS]` 和 `[SEP]`。这两个token的出现，是因为我们调用的模型，在pre-train阶段使用了它们，所以tokenizer也会使用。

不同的模型使用的special tokens不一定相同，所以一定要让tokenizer跟model保持一致！